### Build a SuperDove Constellation in Sedaro

This notebook exercises functionality of deleting any agents with outdated TLEs in the [DEMO] Planet SuperDove Scenario and repopulating the scenario with new agents using current TLEs through the Sedaro API.


In [ ]:
import json
import requests

from sedaro import SedaroApiClient

#### Important: Read Before Running

This notebook makes changes to a scenario branche indicated in the settings section. Ensure any changes to the target branches are saved prior to running this code. Sedaro recommends committing current work and creating new branches in the target repositories to avoid loss of work.

This notebook also requires that you have previously generated an API key in the web UI. That key should be stored in a file called `secrets.json` in the same directory as this notebook with the following format:

```json
{
  "API_KEY": "<API_KEY>"
}
```

API keys grant full access to your repositories and should never be shared. If you think your API key has been compromised, you can revoke it in the user settings interface on the Sedaro website.


In [ ]:
# Settings
with open('../secrets.json', 'r') as file:
    API_KEY = json.load(file)['API_KEY']

with open('../config.json', 'r') as file:
    config = json.load(file)

# Obtain these IDs from the branch list within each repository and add to config.json
SCENARIO_BRANCH_ID = config['SUPERDOVE']['SCENARIO_BRANCH_ID']      # ID of the scenario branch
HOST = config['HOST']                                               # Sedaro instance URL
BASE_AGENT_ID = 'NT-KoZFSELKK8eomP3lkV'

#### Instantiate Client

Instantiate the `SedaroApiClient` with our `API_KEY` and `HOST`, and fetch the desired branch.


In [ ]:
sedaro = SedaroApiClient(api_key=API_KEY, host=HOST)

scenario_branch = sedaro.scenario(SCENARIO_BRANCH_ID)

### Delete satelllites with old TLEs

This is a cleanup step to remove any satellites that were created in a previous run of this notebook. This is not necessary if you are running this notebook for the first time.


In [ ]:
delete_ids = []
for agent_id in scenario_branch.data['index']['TemplatedAgent']:
    if agent_id != BASE_AGENT_ID:
        delete_ids.append(agent_id)

if not delete_ids:
    print('No blocks were deleted')
else:
    scenario_branch.crud(delete=delete_ids)
    print(f'Deleted {len(delete_ids)} spacecraft digital twin agents.')

### Fetch Constellation TLEs


In [ ]:
# Load TLE file
url = 'https://ephemerides.planet-labs.com/planet_mc.tle'
tles = requests.get(url).text.splitlines()

### Create new agents with new TLEs


In [ ]:
base = scenario_branch.Agent.get(BASE_AGENT_ID)
base_orbit = scenario_branch.Orbit.get(base.kinematics)


def create_orbit(tle, orbit_ref_id):
    _orbit = {
        **base_orbit.data,
        'initialStateDefParams': {'tle': tle},
        'id': orbit_ref_id
    }
    return _orbit


def create_agent(name, orbit_ref_id):
    if name[0] == '0':  # Strip line number
        name = name[2:]
    _agent = {**base.data, 'kinematics': orbit_ref_id, 'name': name}
    del _agent['id']
    return _agent


n_sats = int(len(tles)/3)

scenario_branch.crud(
    blocks=[
        create_orbit(tles[3*idx+1] + '\n' + tles[3*idx+2], f'$-{idx}') for idx in range(n_sats)
    ] + [
        create_agent(tles[3*idx], f'$-{idx}') for idx in range(n_sats)
    ]
)
print(f'Created {n_sats} orbits from TLEs and {n_sats} spacecraft digital twin agents.')